In [1]:
from qtyaccounting import qtytools as qts

In [2]:
from qtyaccounting import mftools as mts

In [3]:
import sys
print(sys.path)

['c:\\Users\\iery2\\GitHub\\QTYAccounting', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\python39.zip', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\DLLs', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\lib', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39', '', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\lib\\site-packages', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\lib\\site-packages\\win32', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\lib\\site-packages\\win32\\lib', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\lib\\site-packages\\Pythonwin', 'c:\\Users\\iery2\\anaconda3\\envs\\inventory39\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\iery2\\.ipython']


In [4]:
filename = "example/商品の仕入と売上.txt"
with open(filename, 'r',encoding='utf-8') as f:
    journal = f.read()

In [5]:
print(journal)

商品の仕入と売上の例

<<2022-01-01 &KIND::OPENING ##開始仕訳
Dr　預金 93,000円
Dr　商品#Tシャツ *10個 7,000円
Cr　資本金 100,000>>

<<2022-05-14 ##商品の仕入１
Dr　商品#Tシャツ *10個 6000円
Cr　預金 6000>>

<<2022-08-15 ##商品の仕入２
Dr　商品 #Tシャツ *10個 7000円 
Cr　預金 7000>>

<<2022-10-16 ##商品の売上
Dr 売上原価　#Tシャツ　?E円  $得意先A
Cr 商品 #Tシャツ　*5個 ? ##払出価額の自動計算>>

<<2022-12-16 $得意先A
Dr 売掛金 ?E円
Cr 売上高　#Tシャツ @200 *5個 &天気::晴れ &気温:26度　##売上計上>>


In [6]:
#仕訳ををTree形式にしてから
journal_tree = qts.QTYJournalToTree().translate(journal)
#総勘定元帳を作成
ledgers =qts.InterpretJournalTree().get_ledgers(journal_tree)

In [7]:
df = ledgers.simple_ledger_to_df("商品","","Tシャツ")
df

,entry_id,datetime,quantity,quantity_unit,amount,amount_unit,dr_quantity,dr_amount,cr_quantity,cr_amount,remarks
0,0,2022-01-01,10,個,7000,円,10.0,7000.0,NaN,NaN,開始仕訳開始仕訳
1,1,2022-05-14,10,個,6000,円,10.0,6000.0,NaN,NaN,商品の仕入１商品の仕入１
2,2,2022-08-15,10,個,7000,円,10.0,7000.0,NaN,NaN,商品の仕入２商品の仕入２
3,3,2022-10-16,5,個,OP_AUTO_AMOUNT,NaN,NaN,NaN,5.0,3500.0,払出価額の自動計算商品の売上


In [8]:
ledgers.simple_ledger_to_excel("商品","","Tシャツ")

In [9]:
df_tb = ledgers.tb_to_df(start_datetime="2022-05-01",end_datetime="2023-01-01")
df_tb

,account,sub_account,item,start_quantity,start_amount,sum_dr_quantity,sum_dr_amount,sum_cr_quantity,sum_cr_amount,end_quantity,end_amount
0,預金,,,1,93000,0,0.0,2,13000.0,-1,80000.0
1,売掛金,,,0,0,1,1000.0,0,0.0,1,1000.0
2,商品,,Tシャツ,10,7000,20,13000.0,5,3500.0,25,16500.0
3,資本金,,,1,100000,0,0.0,0,0.0,1,100000.0
4,売上高,,Tシャツ,0,0,0,0.0,5,1000.0,5,1000.0
5,売上原価,,Tシャツ,0,0,1,3500.0,0,0.0,1,3500.0


In [10]:
ledgers.tb_to_excel(start_datetime="2022-05-01",end_datetime="2023-01-01")

filename C:\Users\iery2\GitHub\QTYAccounting\TB.xlsx


In [11]:
lgs_df =ledgers.get_record_df()
lgs_df

,entry_id,counts_dr_cr,order_id,line_no,datetime,account,sub_account,item,partner,person_in_charge,...,amount,amount_unit,dr_quantity,dr_amount,cr_quantity,cr_amount,remarks,KIND,天気,気温
0,0,3,0,0,2022-01-01,預金,,,NaN,NaN,...,93000,円,1.0,93000.0,NaN,NaN,開始仕訳開始仕訳,OPENING,NaN,NaN
1,0,3,2,0,2022-01-01,資本金,,,NaN,NaN,...,100000,NaN,NaN,NaN,1.0,100000.0,開始仕訳開始仕訳,OPENING,NaN,NaN
2,0,3,1,1,2022-01-01,商品,,Tシャツ,NaN,NaN,...,7000,円,10.0,7000.0,NaN,NaN,開始仕訳開始仕訳,OPENING,NaN,NaN
3,1,2,0,0,2022-05-14,商品,,Tシャツ,NaN,NaN,...,6000,円,10.0,6000.0,NaN,NaN,商品の仕入１商品の仕入１,NaN,NaN,NaN
4,1,2,1,0,2022-05-14,預金,,,NaN,NaN,...,6000,NaN,NaN,NaN,1.0,6000.0,商品の仕入１商品の仕入１,NaN,NaN,NaN
5,2,2,0,0,2022-08-15,商品,,Tシャツ,NaN,NaN,...,7000,円,10.0,7000.0,NaN,NaN,商品の仕入２商品の仕入２,NaN,NaN,NaN
6,2,2,1,0,2022-08-15,預金,,,NaN,NaN,...,7000,NaN,NaN,NaN,1.0,7000.0,商品の仕入２商品の仕入２,NaN,NaN,NaN
7,3,2,0,0,2022-10-16,売上原価,,Tシャツ,得意先A,NaN,...,OP_EQUAL_AMOUNT,円,1.0,3500.0,NaN,NaN,商品の売上商品の売上,NaN,NaN,NaN
8,3,2,1,0,2022-10-16,商品,,Tシャツ,NaN,NaN,...,OP_AUTO_AMOUNT,NaN,NaN,NaN,5.0,3500.0,払出価額の自動計算商品の売上,NaN,NaN,NaN
9,4,2,0,0,2022-12-16,売掛金,,,得意先A,NaN,...,OP_EQUAL_AMOUNT,円,1.0,1000.0,NaN,NaN,,NaN,NaN,NaN


In [12]:
# 例：天気や気温と売上数量、売上高との関係
lgs_df[lgs_df["account"]=="売上高"][["天気","気温","cr_quantity","cr_amount"]]

,天気,気温,cr_quantity,cr_amount
10,晴れ,26.0,5.0,1000.0


In [13]:
journal_dic1=qts.QTYJournalTreeToDic().transform(journal_tree)

In [14]:
print(journal_dic1)

{'journal': ['商品の仕入と売上の例', {'journal_entry': {'entry_header': {'datetime': ('DATETIME', '2022-01-01'), 'memo': {'KIND': 'OPENING'}, 'remarks': '開始仕訳'}, 'body': [{'debit': {'account': '預金', 'amount': 93000, 'amount_unit': '円'}}, {'debit': {'account': '商品', 'item': 'Tシャツ', 'quantity': 10, 'quantity_unit': '個', 'amount': 7000, 'amount_unit': '円'}}, {'credit': {'account': '資本金', 'amount': 100000}}], 'entry_footer': {}}}, {'journal_entry': {'entry_header': {'datetime': ('DATETIME', '2022-05-14'), 'remarks': '商品の仕入１'}, 'body': [{'debit': {'account': '商品', 'item': 'Tシャツ', 'quantity': 10, 'quantity_unit': '個', 'amount': 6000, 'amount_unit': '円'}}, {'credit': {'account': '預金', 'amount': 6000}}], 'entry_footer': {}}}, {'journal_entry': {'entry_header': {'datetime': ('DATETIME', '2022-08-15'), 'remarks': '商品の仕入２'}, 'body': [{'debit': {'account': '商品', 'item': 'Tシャツ', 'quantity': 10, 'quantity_unit': '個', 'amount': 7000, 'amount_unit': '円'}}, {'credit': {'account': '預金', 'amount': 7000}}], 'entry_

In [15]:
qts.QTYJournalTreeToDic().save_json(journal_tree,"QTYjournalDic.json")

In [16]:
#辞書として読み込む
journal_dic1 = qts.QTYJournalDicToTree().load_json("QTYjournalDic.json")
#treeへ変換
journal_tree1 = mts.QTYJournalDicToTree().translate(journal_dic1)

In [17]:
#あるいは、jsonを直接Treeへ変換
journal_tree1 = qts.QTYJournalDicToTree().load_json_to_tree("QTYjournalDic.json")

In [21]:
# 1行目
with open("example/mfsample.csv", "r", encoding='shift_jis') as file:
	file_data = file.readline()
print(file_data)
# 摘要欄
import csv
with open('example/mfsample.csv', 'r',encoding='shift_jis') as f:
    for row in csv.DictReader(f):
        print(f"{row['MF仕訳タイプ']} {row['取引日']} {row['借方勘定科目']} {row['借方金額(円)']}/ {row['貸方勘定科目']} {row['貸方金額(円)']} {row['摘要']}")

"取引No","取引日","借方勘定科目","借方補助科目","借方部門","借方取引先","借方税区分","借方インボイス","借方金額(円)","借方税額","貸方勘定科目","貸方補助科目","貸方部門","貸方取引先","貸方税区分","貸方インボイス","貸方金額(円)","貸方税額","摘要","仕訳メモ","タグ","MF仕訳タイプ","決算整理仕訳","作成日時","作成者","最終更新日時","最終更新者"

開始仕訳 2023/04/01 普通預金 93000/ 資本金 100000 
開始仕訳 2023/04/01 商品 7000/  0 <<Dr #Tシャツ *10個>>
 2023/08/01 商品 6000/ 普通預金 6000 <<Dr #Tシャツ *10個>>
 2023/08/02 商品 7000/ 普通預金 7000 <<Dr #Tシャツ *10個>>
 2023/08/03 仕入高 1/ 商品 1 <<Dr #Tシャツ　?E個  $得意先A Cr #Tシャツ　*5個 ? >>
 2023/08/03 売掛金 1/ 売上高 1 <<Dr ?E Cr #Tシャツ @200 *5個 &天気::晴れ &気温:26度　##売上計上>>


In [26]:
#journal_dic に変換
#mf_csv -> csv_tree　-> journal_dic
csv_tree = mts.MfCSVToCSVTree().translate_file("example/mfsample.csv")
journal_dic = mts.CSVTreeToJournalDic().visit(csv_tree)
print(journal_dic)

{'journal': [{'journal_entry': {'entry_header': {'entry_id': '1', 'datetime': '2023-04-01', 'memo': {'取引No': '1', 'KIND': 'OPENING', '作成者': '中谷 賢一', '最終更新者': '中谷 賢一'}, 'remarks': ''}, 'body': [{'debit': {'account': '普通預金', 'sub_account': None, 'amount': 93000, 'memo': {'借方税区分': '対象外', 'ORIGINAL_AMOUNT': 93000, 'ORIGINAL_AMOUNT_UNIT': '円', '借方税額': 0, '借方税額単位': '円'}, 'remarks': ''}}, {'credit': {'account': '資本金', 'sub_account': None, 'amount': 100000, 'memo': {'貸方税区分': '対象外', 'ORIGINAL_AMOUNT': 100000, 'ORIGINAL_AMOUNT_UNIT': '円', '貸方税額': 0, '貸方税額単位': '円'}, 'remarks': ''}}, {'debit': {'account': '商品', 'sub_account': None, 'amount': 7000, 'memo': {'借方税区分': '対象外', 'ORIGINAL_AMOUNT': 7000, 'ORIGINAL_AMOUNT_UNIT': '円', '借方税額': 0, '借方税額単位': '円'}, 'item': 'Tシャツ', 'quantity': 10, 'quantity_unit': '個', 'remarks': ''}}], 'entry_footer': {}}}, {'journal_entry': {'entry_header': {'entry_id': '2', 'datetime': '2023-08-01', 'memo': {'取引No': '2', '作成日時': '2023_8_1_23_02_23', '作成者': '中谷 賢一', '最終更新日時': 

In [30]:
# journal_dicから元帳を作成して分析
#journal_dic -> journal_tree -> ledgers
journal_tree = mts.QTYJournalDicToTree().translate(journal_dic)
#print(journal_tree)
ledgers =qts.InterpretJournalTree().get_ledgers(journal_tree)

In [31]:
journal_tree

Tree(Token('RULE', 'journal'), [Tree(Token('RULE', 'journal_entry'), [Tree(Token('RULE', 'entry_header'), [Tree(Token('RULE', 'datetime'), [Token('DATETIME', '2023-04-01')]), Tree(Token('RULE', 'param_pair'), [Tree(Token('RULE', 'param_mark'), [Token('MEMO_MARK', '&')]), Tree(Token('RULE', 'param'), [Tree(Token('RULE', 'memo_string'), [Tree(Token('RULE', 'key'), [Token('STRING', '取引No')]), Token('STRING', '1')])])]), Tree(Token('RULE', 'param_pair'), [Tree(Token('RULE', 'param_mark'), [Token('MEMO_MARK', '&')]), Tree(Token('RULE', 'param'), [Tree(Token('RULE', 'memo_string'), [Tree(Token('RULE', 'key'), [Token('STRING', 'KIND')]), Token('STRING', 'OPENING')])])]), Tree(Token('RULE', 'param_pair'), [Tree(Token('RULE', 'param_mark'), [Token('MEMO_MARK', '&')]), Tree(Token('RULE', 'param'), [Tree(Token('RULE', 'memo_string'), [Tree(Token('RULE', 'key'), [Token('STRING', '作成者')]), Token('STRING', '中谷 賢一')])])]), Tree(Token('RULE', 'param_pair'), [Tree(Token('RULE', 'param_mark'), [Token('M

In [27]:
# 試算表（dataFrame形式）
# tb from ledgers
df_tb = ledgers.tb_to_df(start_datetime="2022-05-01",end_datetime="2023-01-01")
df_tb

,account,sub_account,item,start_quantity,start_amount,sum_dr_quantity,sum_dr_amount,sum_cr_quantity,sum_cr_amount,end_quantity,end_amount
0,売掛金,,,0,0,0,0,0,0,0,0
1,商品,,Tシャツ,10,7000,0,0,0,0,10,7000
2,資本金,,,1,100000,0,0,0,0,1,100000
3,売上高,,Tシャツ,0,0,0,0,0,0,0,0
4,普通預金,,,1,93000,0,0,0,0,1,93000
5,仕入高,,Tシャツ,0,0,0,0,0,0,0,0


In [25]:
# 試算表（Excel形式）
# ファイル名を指定しない場合、TB.xlsx として保存されます。
ledgers.tb_to_excel(start_datetime="2022-05-01",end_datetime="2023-01-01")

filename C:\Users\iery2\GitHub\QTYAccounting\TB.xlsx


In [28]:
lgs_df =ledgers.get_record_df()
lgs_df

,entry_id,counts_dr_cr,order_id,line_no,datetime,account,sub_account,item,partner,person_in_charge,...,最終更新者,借方税区分,ORIGINAL_AMOUNT_UNIT,借方税額単位,貸方税区分,貸方税額単位,作成日時,最終更新日時,天気,気温
0,0,3,0,0,2023-04-01,普通預金,,,NaN,NaN,...,中谷 賢一,対象外,円,円,NaN,NaN,NaN,NaN,NaN,NaN
1,0,3,1,0,2023-04-01,資本金,,,NaN,NaN,...,中谷 賢一,NaN,円,NaN,対象外,円,NaN,NaN,NaN,NaN
2,0,3,2,1,2023-04-01,商品,,Tシャツ,NaN,NaN,...,中谷 賢一,対象外,円,円,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,0,0,2023-08-01,商品,,Tシャツ,NaN,NaN,...,中谷 賢一,対象外,円,円,NaN,NaN,2023_8_1_23_02_23,2023_8_1_23_04_10,NaN,NaN
4,1,2,1,0,2023-08-01,普通預金,,,NaN,NaN,...,中谷 賢一,NaN,円,NaN,対象外,円,2023_8_1_23_02_23,2023_8_1_23_04_10,NaN,NaN
5,2,2,0,0,2023-08-02,商品,,Tシャツ,NaN,NaN,...,中谷 賢一,対象外,円,円,NaN,NaN,2023_8_1_23_03_42,2023_8_1_23_04_23,NaN,NaN
6,2,2,1,0,2023-08-02,普通預金,,,NaN,NaN,...,中谷 賢一,NaN,円,NaN,対象外,円,2023_8_1_23_03_42,2023_8_1_23_04_23,NaN,NaN
7,3,4,0,0,2023-08-03,仕入高,,Tシャツ,NaN,NaN,...,中谷 賢一,課税仕入_10_PCT_,円,円,NaN,NaN,2023_8_1_23_11_46,2023_8_1_23_12_55,NaN,NaN
8,3,4,1,0,2023-08-03,商品,,Tシャツ,NaN,NaN,...,中谷 賢一,NaN,円,NaN,対象外,円,2023_8_1_23_11_46,2023_8_1_23_12_55,NaN,NaN
9,3,4,2,1,2023-08-03,売掛金,,,NaN,NaN,...,中谷 賢一,対象外,円,円,NaN,NaN,2023_8_1_23_11_46,2023_8_1_23_12_55,NaN,NaN


In [29]:
lgs_df.columns

Index(['entry_id', 'counts_dr_cr', 'order_id', 'line_no', 'datetime',
       'account', 'sub_account', 'item', 'partner', 'person_in_charge', 'memo',
       'memo_str', 'memo_number', 'memo_number_unit', 'price', 'quantity',
       'quantity_unit', 'amount', 'amount_unit', 'dr_quantity', 'dr_amount',
       'cr_quantity', 'cr_amount', 'remarks', '取引No', 'KIND', '作成者', '最終更新者',
       '借方税区分', 'ORIGINAL_AMOUNT_UNIT', '借方税額単位', '貸方税区分', '貸方税額単位', '作成日時',
       '最終更新日時', '天気', '気温'],
      dtype='object')